<a href="https://colab.research.google.com/github/Dltmd202/DeepLearningStudy/blob/main/MultiCalssNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>